##Using HackaMol to calculate the Ramachandran Plot

In [1]:
use Modern::Perl;
use HackaMol;

### Load a molecule within multiple chains from the pdbid

In [2]:
my $pdbid    = '1AXC';
my $mol      = HackaMol->new->pdbid_mol("$pdbid");
my @bb_atoms = $mol->select_atoms(sub{
                                        $_ -> name eq 'C' or
                                        $_ -> name eq 'CA' or
                                        $_ -> name eq 'N'
                                     });
                                     

Warning: 1AXC.pdb exists, set self->overwrite(1) to overwrite at /Users/chem_student/perl5/perlbrew/perls/perl-5.20.1/lib/site_perl/5.20.1/HackaMol.pm line 26.


## Create groups of atoms according to their chains

In [3]:
my @chains = HackaMol->new->group_by_atom_attr('chain',@bb_atoms);
my $bb_mol = HackaMol::Molecule->new(groups => [@chains]);
$bb_mol->print_pdb ("$pdbid\_backbone.pdb");1

1


Warning: overwrite 1AXC_backbone.pdb at reply input line 3.


##Construct the backbone dihedrals for each chain
The backbone atoms are ordered in repeats {N, CA, C}. Walking down the backbone: N0-CA0-C0-N1-CA1-C1-N2-CA2-C2 ...there will be dihedrals about the CA0-C0(PSI), C0-N1(OMEGA), N1-CA1 (PHI), CA1-C1 (PSI), ... bonds.  Since there are no bonds between chains, they must be treated separately.

In [4]:
my @dihedrals;
foreach my $chain ($bb_mol->all_groups){
    push @dihedrals, HackaMol->new->build_dihedrals($chain->all_atoms);
}

In [7]:
my $i= 0;
printf (" %7s %7s %7s \n" , qw(  phi   psi   omega));
my @dihe_deg = map{
                    $dihedrals[$_]->dihe_deg
} 0 .. $#dihedrals;

unshift @dihe_deg, 0;

while ($i+3 < $#dihe_deg+4){
    printf ("%5i %7.2f %7.2f %7.2f \n", $i, map{$dihe_deg[$_]}($i, $i+1, $i+2));
  if($dihe_deg[$i] > 170 and $dihe_deg[$i+1] > 130){
        # printf ("shit %5i %7.2f %7.2f %7.2f \n", $i, map{$dihe_deg[$_]}($i, $i+1, $i+2));
         #   $dihedrals[$i]->print_pdb;
       # $dihedrals[$i+1]->print_pdb;
  }
    $i += 3;
} 

     phi     psi   omega 
    0    0.00  145.59 -179.14 
    3 -113.76  132.18  177.82 
    6 -154.09  113.74  178.65 
    9 -122.23  146.10 -178.99 
   12 -128.64  130.03 -179.93 
   15 -123.32  109.01  179.92 
   18  -62.51  -32.29  178.43 
   21  -92.14   90.63  179.66 
   24  -57.88  -41.28 -179.47 
   27  -47.63  -46.45 -178.89 
   30  -46.11  -48.02 -178.69 
   33  -73.52  -30.63  178.28 
   36  -64.67  -54.44  178.98 
   39  -59.07  -39.24  179.61 
   42  -60.86  -42.64 -178.30 
   45  -73.12  -34.05  178.46 
   48  -72.61  -23.59  178.57 
   51  -79.42  -21.50 -178.71 
   54 -104.92  -48.04 -177.34 
   57  -36.14  -36.57 -178.96 
   60  -84.57  -27.98  178.83 
   63  -86.03  -43.66  178.62 
   66 -123.58  130.24 -178.83 
   69  -89.77  -43.77  179.93 
   72 -123.01  143.04 -179.82 
   75 -165.20  150.54  179.78 
   78  -88.20  131.29  178.15 
   81 -105.65  106.80  179.68 
   84  -89.39  118.93  179.51 
   87 -105.56  133.03 -179.53 
   90 -152.05  164.87 -179.18 
   93  -51.97

Warning: Use of uninitialized value in printf at reply input line 10.

Use of uninitialized value in printf at reply input line 10.
